In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [29]:
#week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Jan_list = [2, 9, 16, 23, 30]

In [30]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [31]:
len(link_day_minute_Jan_list)

14400

In [32]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [33]:
np.size(x, 0), np.size(x, 1)

(24, 600)

In [34]:
x[:,:2]

matrix([[ 1235.03209639,  1275.32760486],
        [ 1202.83746161,  1200.99906373],
        [ 1554.94634607,  1537.80960515],
        [ 1543.14090375,  1533.71667045],
        [ 1506.41819308,  1462.58788742],
        [ 1449.97717537,  1489.45836837],
        [ 1250.11071279,  1298.88381015],
        [ 1280.44551751,  1252.15430278],
        [ 1635.03935469,  1726.81615022],
        [ 1660.72933309,  1589.84619802],
        [ 1340.50766712,  1388.30001736],
        [ 1297.24315504,  1245.91501958],
        [ 1183.35724495,  1193.1505411 ],
        [ 1311.71709436,  1395.07401973],
        [ 1135.5074771 ,  1131.68352257],
        [ 1022.26780226,  1028.62000699],
        [ 1266.22053596,  1243.71833892],
        [ 1305.98186902,  1265.41462246],
        [ 1555.58628784,  1544.03827366],
        [ 1558.14461437,  1542.66401738],
        [ 1498.24147246,  1468.86731119],
        [ 1347.94716816,  1412.03518371],
        [ 1245.36168302,  1238.48285715],
        [ 1249.83146055,  1238.233

In [35]:
np.size(A,0), np.size(A,1)

(24, 140)

In [36]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e+01, 2e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 25
 AA' NZ     : 3.000e+02
 Factor NZ  : 3.250e+02
 Factor Ops : 5.525e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.20996405e+07  0.00000000e+00  8.87e+03 1.03e+03  1.00e+06     0s
   1  -6.32334585e+06 -8.57943979e+06  7.54e+02 8.76e+01  1.82e+05     0s
   2  -4.64081449e+06 -5.78430352e+06  2.67e+01 3.11e+00  2.41e+04     0s
   3  -4.71170049e+06 -4.80087466e+06  1.31e-01 1.52e-02  1.6

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [38]:
lam_list

[6.198007503349369e-13,
 941.5908282467939,
 6.803369850153859e-13,
 0.00010167000214266007,
 1525.8302722171452,
 0.0001694683976373023,
 2.0530436175232184e-10,
 856.582001518788,
 0.0011321351268835725,
 1237.638600652464,
 371.54907114420524,
 5.9784112571234244e-05,
 0.00014738186899986017,
 0.00015620935241639298,
 1497.0705291702388,
 1425.7210426190504,
 0.005566054105685447,
 1017.923982439698,
 6.876848070059248e-05,
 0.00016946333867750955,
 0.0001693811977952226,
 1.1178382646964225e-05,
 1353.6707088032247,
 712.6270857766897,
 203.77123853638628,
 0.08847281503764748,
 0.47850438787474303,
 1134.0478935693573,
 0.011098568092314243,
 0.004733937881660932,
 0.053473019460260396,
 1077.686902133604,
 1102.4288537760692,
 165.71893021920644,
 1100.12157415023,
 1.3750649649156775e-07,
 1.3750705443802988e-07,
 1.3750650629051075e-07,
 0.005845422054461313,
 865.337541225362,
 733.7244947828469,
 0.03600618433891811,
 0.0749397962435765,
 0.07505581841483888,
 984.47821900576